In [215]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
import torch.nn.init as init

# Create the NN model

In [290]:
class ClassificationModel(nn.Module): 
        
    def __init__(self, nodes_per_layer, input_shape, num_classes):
        super(ClassificationModel, self).__init__()

        layers = []
        # input layer with the input dimension of the data shape        
        layers.append(nn.Linear(input_shape, nodes_per_layer[0]))
        # He initialization for the first layer
        init.kaiming_uniform_(layers[0].weight, mode='fan_in', nonlinearity='relu')
        # the activation function for each node
        layers.append(nn.ReLU()) #nn.Tanh() nn.ReLU()
        # hidden layers with dimension of nodes_per_layer
        for i in range(1, len(nodes_per_layer)):
            layers.append(nn.Linear(nodes_per_layer[i - 1], nodes_per_layer[i]))
            # He initialization for the hidden layers
            init.kaiming_uniform_(layers[-1].weight, mode='fan_in', nonlinearity='relu')
            layers.append(nn.ReLU())
        # output layer with the binary output
        layers.append(nn.Linear(nodes_per_layer[-1], num_classes))
        layers.append(nn.Softmax(dim=1))

        self.model = nn.Sequential(*layers)
                
  
    def forward(self, x):
        x = x.to(self.model[0].weight.dtype)
        return self.model(x)
    
    def train_model(self, train_loader, num_epochs=5, lr=0.001, val_loader=True) :
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(self.parameters(), lr=lr)

        for epoch in range(num_epochs):
            for inputs, labels in train_loader:
                optimizer.zero_grad()
                outputs = self(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

            
            if val_loader is not None:
                self.eval()
                correct = 0
                total = 0
                with torch.no_grad():
                    for val_inputs, val_labels in val_loader:
                        val_outputs = self(val_inputs)
                        _, predicted = torch.max(val_outputs.data, 1)
                        total += val_labels.size(0)
                        correct += (predicted == val_labels).sum().item()

                accuracy = correct / total
            
            print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}, Validation Accuracy: {accuracy}')
            
    def predict(self, input_data):
        self.eval()
        with torch.no_grad():
            outputs = self(input_data)
            _, predicted = torch.max(outputs.data, 1)
        return predicted

In [217]:
def train_test_split(data: pd.DataFrame, target_label : str, test_size=0.2, return_torch=None):
        
    # split the data into train and test
    train = data.sample(frac=(1-test_size),random_state=123)
    test = data.drop(train.index)
    
    # split the train and test into X and Y
    train_X = train.drop([target_label], axis=1).values
    train_Y = train[target_label].values
    test_X = test.drop([target_label], axis=1).values
    test_Y = test[target_label].values
        
    if return_torch:
        train_X = torch.tensor(train_X)
        train_Y = torch.tensor(train_Y)
        test_X = torch.tensor(test_X)
        test_Y = torch.tensor(test_Y)
        
    return train_X, train_Y, test_X, test_Y

# Load the data

In [218]:
# Congression Voting Dataset
cong_voting = pd.read_csv('./preprocessed-datasets/CongressionVoting_prepro.csv')

label2num = {"democrat": 0, "republican": 1}
num2label = {0: "democrat", 1: "republican"}

# convert the target label to numeric
cong_voting["class"] = cong_voting["class"].apply(lambda x: label2num[x])

In [219]:
cong_voting["class"].value_counts()

0    137
1     80
Name: class, dtype: int64

In [220]:
# Wine Quality Dataset
wine_quality = pd.read_csv('./preprocessed-datasets/wine_quality_prepro.csv', index_col=0)
wine_quality.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,class,wine_type
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,1
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1


In [229]:
# Review dataset
review = pd.read_csv('./preprocessed-datasets/Review_prepro.csv')

labels = review["class"].unique()
label2num = {label: idx for idx, label in enumerate(labels)}
num2label = {idx: label for idx, label in enumerate(labels)}

# convert the target label to numeric
review["class"] = review["class"].apply(lambda x: label2num[x])
review.head()

,ID,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V9992,V9993,V9994,V9995,V9996,V9997,V9998,V9999,V10000,class
0,0,14,9,8,12,7,4,5,3,2,...,0,1,0,1,0,1,1,0,0,0
1,1,12,3,8,6,5,3,2,3,2,...,0,0,0,0,0,0,0,0,0,1
2,2,17,5,7,12,8,3,0,3,2,...,0,1,0,0,0,0,1,0,0,2
3,3,9,6,10,6,4,7,7,3,3,...,0,0,0,0,2,0,0,0,0,3
4,4,8,4,5,5,1,7,0,0,0,...,0,0,3,0,0,0,0,0,0,4


In [230]:
review["class"].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
      dtype=int64)

## Run the model

In [300]:
# run the model for cong_voting dataset
train_X, train_Y, test_X, test_Y = train_test_split(cong_voting, "class", return_torch=True)

nodes_per_layer = [30, 30, 30]
input_shape = train_X.shape[1]
num_classes = 2 # binary classification
num_samples = train_X.shape[0] 

dataset = TensorDataset(train_X, train_Y) # wrap X and Y into a single dataset
train_loader = DataLoader(dataset, batch_size=32, shuffle=True) 
# in batch we have 32 samples, and we shuffle the data after the iteration over each batch
model = ClassificationModel(nodes_per_layer, input_shape, num_classes)
model.train_model(train_loader, num_epochs= 25, lr= 0.0001, val_loader=train_loader)
Y_pred = model.predict(test_X)
print(f"The f1 score is {metrics.f1_score(test_Y, Y_pred)}")

Epoch 1/25, Loss: 0.898162841796875, Validation Accuracy: 0.6206896551724138
Epoch 2/25, Loss: 0.7804933786392212, Validation Accuracy: 0.6206896551724138
Epoch 3/25, Loss: 0.6891213655471802, Validation Accuracy: 0.6206896551724138
Epoch 4/25, Loss: 0.7292973399162292, Validation Accuracy: 0.6206896551724138
Epoch 5/25, Loss: 0.672873318195343, Validation Accuracy: 0.6206896551724138
Epoch 6/25, Loss: 0.7340871691703796, Validation Accuracy: 0.6206896551724138
Epoch 7/25, Loss: 0.7242613434791565, Validation Accuracy: 0.6206896551724138
Epoch 8/25, Loss: 0.9217442870140076, Validation Accuracy: 0.6379310344827587
Epoch 9/25, Loss: 0.688845157623291, Validation Accuracy: 0.6896551724137931
Epoch 10/25, Loss: 0.6827824711799622, Validation Accuracy: 0.6436781609195402
Epoch 11/25, Loss: 0.586604654788971, Validation Accuracy: 0.6494252873563219
Epoch 12/25, Loss: 0.543513834476471, Validation Accuracy: 0.6494252873563219
Epoch 13/25, Loss: 0.7503235936164856, Validation Accuracy: 0.6551

In [307]:
# run the model for wine_quality dataset
train_X, train_Y, test_X, test_Y = train_test_split(wine_quality, "wine_type", return_torch=True)

nodes_per_layer = [20, 20, 20,20]
input_shape = train_X.shape[1]
num_classes = 10 # binary classification
num_samples = train_X.shape[0] 

dataset = TensorDataset(train_X, train_Y)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)
model = ClassificationModel(nodes_per_layer, input_shape, num_classes)
model.train_model(train_loader, lr= 0.001, val_loader=train_loader, num_epochs= 25)
Y_pred = model.predict(test_X)
print(f"The f1 score is {metrics.f1_score(test_Y, Y_pred, average='micro')}")

Epoch 1/25, Loss: 1.4615298509597778, Validation Accuracy: 0.9378607156598692
Epoch 2/25, Loss: 1.5346012115478516, Validation Accuracy: 0.9253559061177375
Epoch 3/25, Loss: 1.6040185689926147, Validation Accuracy: 0.9392073874567142
Epoch 4/25, Loss: 1.4611499309539795, Validation Accuracy: 0.9328587918430166
Epoch 5/25, Loss: 1.5327692031860352, Validation Accuracy: 0.9438245479030396
Epoch 6/25, Loss: 1.4613572359085083, Validation Accuracy: 0.9378607156598692
Epoch 7/25, Loss: 1.5344102382659912, Validation Accuracy: 0.9411312043093497
Epoch 8/25, Loss: 1.5338350534439087, Validation Accuracy: 0.9445940746440938
Epoch 9/25, Loss: 1.5326082706451416, Validation Accuracy: 0.9442093112735668
Epoch 10/25, Loss: 1.5065284967422485, Validation Accuracy: 0.9270873412851096
Epoch 11/25, Loss: 1.5302140712738037, Validation Accuracy: 0.9444016929588304
Epoch 12/25, Loss: 1.461670994758606, Validation Accuracy: 0.9488264717198923
Epoch 13/25, Loss: 1.665830373764038, Validation Accuracy: 0.9

In [247]:
# run the model for wine_quality dataset
train_X, train_Y, test_X, test_Y = train_test_split(review, "class", return_torch=True)

nodes_per_layer = [30,30,30,40]
input_shape = train_X.shape[1]
num_classes = 50 # binary classification
num_samples = train_X.shape[0] 

dataset = TensorDataset(train_X, train_Y)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)
model = ClassificationModel(nodes_per_layer, input_shape, num_classes)
model.train_model(train_loader, lr= 0.0001, val_loader=train_loader, num_epochs= 100)
Y_pred = model.predict(test_X)
print(f"The f1 score is {metrics.f1_score(test_Y, Y_pred, average='micro')}")

Epoch 1/100, Loss: 3.909952402114868, Validation Accuracy: 0.023333333333333334
Epoch 2/100, Loss: 3.9067938327789307, Validation Accuracy: 0.023333333333333334
Epoch 3/100, Loss: 3.908646821975708, Validation Accuracy: 0.023333333333333334
Epoch 4/100, Loss: 3.9057273864746094, Validation Accuracy: 0.023333333333333334
Epoch 5/100, Loss: 3.9008371829986572, Validation Accuracy: 0.023333333333333334
Epoch 6/100, Loss: 3.8950204849243164, Validation Accuracy: 0.028333333333333332
Epoch 7/100, Loss: 3.884312868118286, Validation Accuracy: 0.043333333333333335
Epoch 8/100, Loss: 3.900156259536743, Validation Accuracy: 0.06666666666666667
Epoch 9/100, Loss: 3.925274610519409, Validation Accuracy: 0.06666666666666667
Epoch 10/100, Loss: 3.871370315551758, Validation Accuracy: 0.07333333333333333
Epoch 11/100, Loss: 3.889446258544922, Validation Accuracy: 0.07333333333333333
Epoch 12/100, Loss: 3.9183194637298584, Validation Accuracy: 0.075
Epoch 13/100, Loss: 3.8231916427612305, Validation 

In [67]:
model = ClassificationModel(nodes_per_layer, input_shape, num_classes)
print(model)

ClassificationModel(
  (model): Sequential(
    (0): Linear(in_features=12, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=7, bias=True)
    (3): ReLU()
    (4): Linear(in_features=7, out_features=5, bias=True)
    (5): ReLU()
    (6): Linear(in_features=5, out_features=2, bias=True)
    (7): Softmax(dim=1)
  )
)
